# Azure Entra External ID Setup for FHIR Service

This notebook automates the setup of Microsoft Entra External ID to grant access to an Azure Health Data Services FHIR service.

## Overview
- **Primary Tenant**: Your main Azure AD tenant where the FHIR service will be deployed
- **External Tenant**: The Entra External ID tenant that will provide authentication

## Prerequisites
- Azure PowerShell modules installed
- Appropriate permissions in both tenants
- Global Administrator or equivalent role

## Configuration Variables

Set up your environment variables for both tenants.

In [ ]:
# Primary Tenant Configuration (where FHIR service will be deployed)
$primaryTenantId = "<YOUR-PRIMARY-TENANT-ID>"
$subscriptionId = "<YOUR-SUBSCRIPTION-ID>"
$resourceGroupName = "<YOUR-RESOURCE-GROUP-NAME>"
$region = "<YOUR-REGION>"  # e.g., "eastus"
$workspaceName = "<YOUR-WORKSPACE-NAME>"
$fhirServiceName = "<YOUR-FHIR-SERVICE-NAME>"

# External Tenant Configuration (will be populated after creation)
$externalTenantId = "<EXTERNAL-TENANT-ID>"  # Fill after Step 1
$externalTenantName = "<EXTERNAL-TENANT-NAME>"  # e.g., "myexternalid"
$externalTenantDomain = "<EXTERNAL-TENANT-DOMAIN>"  # e.g., "myexternalid.onmicrosoft.com"

# App Registration (will be populated after Step 4)
$appRegistrationClientId = "<APP-REGISTRATION-CLIENT-ID>"  # Fill after Step 4
$b2cExtensionAppId = "<B2C-EXTENSIONS-APP-ID>"  # Fill after Step 7

# Test User (will be populated after Step 2)
$testUserObjectId = "<TEST-USER-OBJECT-ID>"  # Fill after Step 2
$testUserPrincipalName = "<TEST-USER-PRINCIPAL-NAME>"  # Fill after Step 2

Write-Host "Configuration variables set. Remember to update these values as you progress through the steps." -ForegroundColor Green

## Step 0: Install Required Modules

Ensure all necessary PowerShell modules are installed.

In [ ]:
# Check and install required modules
$requiredModules = @('Az.Accounts', 'Az.Resources', 'Az.HealthcareApis', 'Microsoft.Graph')

foreach ($module in $requiredModules) {
    if (!(Get-Module -ListAvailable -Name $module)) {
        Write-Host "Installing $module..." -ForegroundColor Yellow
        Install-Module -Name $module -Force -AllowClobber -Scope CurrentUser
    } else {
        Write-Host "$module is already installed." -ForegroundColor Green
    }
}

Write-Host "All required modules are ready." -ForegroundColor Green

## Step 1: Create Entra External ID Tenant

**MANUAL STEP**: External ID tenant creation must be done through the Azure Portal.

### Instructions:
1. Go to [https://entra.microsoft.com](https://entra.microsoft.com)
2. Navigate to **Identity** > **Overview** > **Manage tenants**
3. Click **+ Create**
4. Select **External** tenant type
5. Enter:
   - Tenant Name
   - Domain Name
   - Country/Region
6. Select your subscription and resource group
7. Click **Create**
8. Set up MFA when prompted
9. **Copy the following values and update the configuration cell above:**
   - External Tenant ID
   - External Tenant Name
   - External Tenant Domain

After completing this step manually, update the variables in the configuration cell and run the verification below.

In [ ]:
# Verification: Connect to External Tenant
if ($externalTenantId -eq "<EXTERNAL-TENANT-ID>") {
    Write-Host "⚠️  Please update the external tenant configuration variables above before proceeding." -ForegroundColor Yellow
} else {
    try {
        Connect-AzAccount -Tenant $externalTenantId
        $context = Get-AzContext
        Write-Host "✓ Successfully connected to External Tenant: $($context.Tenant.Id)" -ForegroundColor Green
        Write-Host "  Account: $($context.Account.Id)" -ForegroundColor Cyan
    } catch {
        Write-Host "✗ Failed to connect to external tenant: $_" -ForegroundColor Red
    }
}

## Step 2: Create Test User in External Tenant

Create a test user in the External ID tenant using Microsoft Graph.

In [ ]:
# Connect to Microsoft Graph for External Tenant
Connect-MgGraph -TenantId $externalTenantId -Scopes "User.ReadWrite.All"

# Generate a random password
$passwordProfile = @{
    Password = "TempPassword123!@#"
    ForceChangePasswordNextSignIn = $true
}

# Create test user
$testUser = @{
    AccountEnabled = $true
    DisplayName = "Test Patient1"
    MailNickname = "testpatient1"
    UserPrincipalName = "testpatient1@$externalTenantDomain"
    PasswordProfile = $passwordProfile
}

try {
    $user = New-MgUser -BodyParameter $testUser
    Write-Host "✓ Test user created successfully" -ForegroundColor Green
    Write-Host "  User Principal Name: $($user.UserPrincipalName)" -ForegroundColor Cyan
    Write-Host "  Object ID: $($user.Id)" -ForegroundColor Cyan
    Write-Host "  Temporary Password: TempPassword123!@#" -ForegroundColor Yellow
    Write-Host "\n⚠️  Update the testUserObjectId and testUserPrincipalName variables in the configuration cell" -ForegroundColor Yellow
} catch {
    Write-Host "✗ Failed to create user: $_" -ForegroundColor Red
}

In [ ]:
# Verification: Check if user exists
try {
    $user = Get-MgUser -UserId $testUserObjectId
    Write-Host "✓ User verified:" -ForegroundColor Green
    Write-Host "  Display Name: $($user.DisplayName)" -ForegroundColor Cyan
    Write-Host "  UPN: $($user.UserPrincipalName)" -ForegroundColor Cyan
    Write-Host "  Object ID: $($user.Id)" -ForegroundColor Cyan
} catch {
    Write-Host "✗ User verification failed: $_" -ForegroundColor Red
}

## Step 3: Create fhirUser Custom Attribute and User Flow

**SEMI-MANUAL STEP**: Custom attributes and user flows need to be created through the Azure Portal.

### Instructions:

#### 3A: Create Custom Attribute
1. In Azure Portal, search for **External Identities**
2. Navigate to **Custom user attributes**
3. Click **+ Add**
4. Enter:
   - Name: `fhirUser`
   - Data type: String
   - Description: "The fully qualified FHIR resource Id associated with the user (e.g. Patient Resource)"
5. Click **Create**

#### 3B: Create User Flow
1. In **External Identities**, navigate to **User flows**
2. Click **+ New user flow**
3. Enter a name for the user flow
4. Select **Email with password**
5. Click **Show more** and select the **fhirUser** attribute
6. Click **Ok** then **Create**
7. **Note the User Flow name** for later use

Run the verification cell below after completing these steps.

In [ ]:
# Verification: List custom attributes (requires Microsoft Graph)
Write-Host "Custom attributes and user flows must be verified through the Azure Portal." -ForegroundColor Yellow
Write-Host "Navigate to External Identities to verify:" -ForegroundColor Cyan
Write-Host "  1. Custom user attributes contains 'fhirUser'" -ForegroundColor Cyan
Write-Host "  2. User flows contains your newly created flow" -ForegroundColor Cyan

## Step 4: Create and Configure Resource Application

Register an application in the External ID tenant to represent the FHIR service.

In [ ]:
# Ensure connected to Microsoft Graph
Connect-MgGraph -TenantId $externalTenantId -Scopes "Application.ReadWrite.All"

# Create app registration
$appRegistration = @{
    DisplayName = "FHIR Service Resource App"
    SignInAudience = "AzureADMyOrg"
    PublicClient = @{
        RedirectUris = @("http://localhost:3000")
    }
}

try {
    $app = New-MgApplication -BodyParameter $appRegistration
    Write-Host "✓ App registration created successfully" -ForegroundColor Green
    Write-Host "  Application Name: $($app.DisplayName)" -ForegroundColor Cyan
    Write-Host "  Application (Client) ID: $($app.AppId)" -ForegroundColor Cyan
    Write-Host "  Object ID: $($app.Id)" -ForegroundColor Cyan
    Write-Host "\n⚠️  Update the appRegistrationClientId variable in the configuration cell with: $($app.AppId)" -ForegroundColor Yellow
    
    # Store for next steps
    $global:appObjectId = $app.Id
    $global:appClientId = $app.AppId
} catch {
    Write-Host "✗ Failed to create app registration: $_" -ForegroundColor Red
}

### Step 4B: Configure OAuth2 Permissions

The OAuth2 permissions need to be configured in the app manifest. This step requires manual configuration.

**MANUAL STEP**:
1. Go to Azure Portal > Entra External ID tenant
2. Navigate to **App registrations** > Your app
3. Click **Manifest**
4. Find the `oauth2PermissionScopes` array
5. Replace it with the permissions from: [oauth2Permissions.json](https://raw.githubusercontent.com/Azure-Samples/azure-health-data-and-ai-samples/main/samples/fhir-aad-b2c/oauth2Permissions.json)
6. Save the manifest

In [ ]:
# Download and display the OAuth2 permissions structure
$permissionsUrl = "https://raw.githubusercontent.com/Azure-Samples/azure-health-data-and-ai-samples/main/samples/fhir-aad-b2c/oauth2Permissions.json"

try {
    $permissions = Invoke-RestMethod -Uri $permissionsUrl
    Write-Host "OAuth2 Permissions structure:" -ForegroundColor Cyan
    $permissions | ConvertTo-Json -Depth 10
    Write-Host "\n⚠️  Copy the above JSON and paste it into the oauth2PermissionScopes array in the app manifest" -ForegroundColor Yellow
} catch {
    Write-Host "✗ Failed to download permissions: $_" -ForegroundColor Red
    Write-Host "Please manually download from: $permissionsUrl" -ForegroundColor Yellow
}

### Step 4C: Expose an API

In [ ]:
# Set Application ID URI
try {
    $appIdUri = "api://$appRegistrationClientId"
    Update-MgApplication -ApplicationId $global:appObjectId -IdentifierUris @($appIdUri)
    Write-Host "✓ Application ID URI set to: $appIdUri" -ForegroundColor Green
} catch {
    Write-Host "✗ Failed to set Application ID URI: $_" -ForegroundColor Red
}

### Step 4D: Configure API Permissions and Admin Consent

**MANUAL STEPS**:
1. Navigate to **API permissions** in your app registration
2. Click **+ Add a permission**
3. Select **APIs my organization uses**
4. Search for and select **FHIR Service**
5. In the **Patient** section, select appropriate permissions
6. Click **Add permissions**
7. Click **Grant admin consent** for your tenant
8. Confirm by clicking **Yes**

### Step 4E: Enable Token Claims Mapping

In [ ]:
# Enable acceptMappedClaims in the manifest
try {
    Update-MgApplication -ApplicationId $global:appObjectId -Api @{ AcceptMappedClaims = $true }
    Write-Host "✓ Token claims mapping enabled (acceptMappedClaims = true)" -ForegroundColor Green
} catch {
    Write-Host "✗ Failed to enable claims mapping: $_" -ForegroundColor Red
}

### Step 4F: Configure SSO Claims

**MANUAL STEPS** (requires Enterprise Application configuration):
1. Navigate to **Enterprise applications** in your External ID tenant
2. Find and select your application
3. Go to **Single sign-on (Preview)**
4. Click **Edit** in Attributes & Claims section
5. Click **+ Add new claim**
6. Configure:
   - Name: `fhirUser`
   - Source: Directory schema extension
   - Application: Select `b2c-extensions-app`
7. In the extension attributes window, select `user.fhirUser`
8. Click **Add** and then **Save**

In [ ]:
# Verification: Get app details
try {
    $app = Get-MgApplication -ApplicationId $global:appObjectId
    Write-Host "✓ App Registration Configuration:" -ForegroundColor Green
    Write-Host "  Display Name: $($app.DisplayName)" -ForegroundColor Cyan
    Write-Host "  Client ID: $($app.AppId)" -ForegroundColor Cyan
    Write-Host "  Application ID URI: $($app.IdentifierUris -join ', ')" -ForegroundColor Cyan
    Write-Host "  Accept Mapped Claims: $($app.Api.AcceptMappedClaims)" -ForegroundColor Cyan
} catch {
    Write-Host "✗ Verification failed: $_" -ForegroundColor Red
}

## Step 5: Deploy FHIR Service with External ID Configuration

Deploy Azure Health Data Services workspace and FHIR service in the PRIMARY tenant.

In [ ]:
# Connect to PRIMARY tenant
Connect-AzAccount -Tenant $primaryTenantId -Subscription $subscriptionId
Set-AzContext -Subscription $subscriptionId

Write-Host "✓ Connected to primary tenant" -ForegroundColor Green
$context = Get-AzContext
Write-Host "  Subscription: $($context.Subscription.Name)" -ForegroundColor Cyan
Write-Host "  Tenant: $($context.Tenant.Id)" -ForegroundColor Cyan

In [ ]:
# Create resource group
try {
    $rg = Get-AzResourceGroup -Name $resourceGroupName -ErrorAction SilentlyContinue
    if ($null -eq $rg) {
        New-AzResourceGroup -Name $resourceGroupName -Location $region
        Write-Host "✓ Resource group created: $resourceGroupName" -ForegroundColor Green
    } else {
        Write-Host "✓ Resource group already exists: $resourceGroupName" -ForegroundColor Green
    }
} catch {
    Write-Host "✗ Failed to create resource group: $_" -ForegroundColor Red
}

In [ ]:
# Deploy FHIR Service with External ID configuration
$smartAuthorityUrl = "https://$externalTenantName.ciamlogin.com/$externalTenantId"
$templateUri = "https://raw.githubusercontent.com/Azure-Samples/azure-health-data-and-ai-samples/main/samples/fhir-aad-b2c/fhir-service-arm-template.json"

Write-Host "Deploying FHIR service with configuration:" -ForegroundColor Cyan
Write-Host "  Workspace: $workspaceName" -ForegroundColor Cyan
Write-Host "  FHIR Service: $fhirServiceName" -ForegroundColor Cyan
Write-Host "  SMART Authority: $smartAuthorityUrl" -ForegroundColor Cyan
Write-Host "  SMART Client ID: $appRegistrationClientId" -ForegroundColor Cyan
Write-Host "\nThis may take several minutes..." -ForegroundColor Yellow

try {
    $deployment = New-AzResourceGroupDeployment `
        -ResourceGroupName $resourceGroupName `
        -TemplateUri $templateUri `
        -tenantid $primaryTenantId `
        -region $region `
        -workspaceName $workspaceName `
        -fhirServiceName $fhirServiceName `
        -smartAuthorityUrl $smartAuthorityUrl `
        -smartClientId $appRegistrationClientId
    
    Write-Host "✓ FHIR service deployed successfully" -ForegroundColor Green
    Write-Host "  Deployment State: $($deployment.ProvisioningState)" -ForegroundColor Cyan
} catch {
    Write-Host "✗ Deployment failed: $_" -ForegroundColor Red
}

In [ ]:
# Verification: Get FHIR service details
try {
    $fhirService = Get-AzHealthcareApisService -ResourceGroupName $resourceGroupName -Name $fhirServiceName
    $fhirUrl = "https://$workspaceName-$fhirServiceName.fhir.azurehealthcareapis.com"
    
    Write-Host "✓ FHIR Service Verified:" -ForegroundColor Green
    Write-Host "  Name: $($fhirService.Name)" -ForegroundColor Cyan
    Write-Host "  Location: $($fhirService.Location)" -ForegroundColor Cyan
    Write-Host "  FHIR URL: $fhirUrl" -ForegroundColor Cyan
    Write-Host "  Provisioning State: $($fhirService.ProvisioningState)" -ForegroundColor Cyan
    
    # Store FHIR URL for later use
    $global:fhirUrl = $fhirUrl
} catch {
    Write-Host "✗ Verification failed: $_" -ForegroundColor Red
}

## Step 6: Load Patient Resource in FHIR Service

Create a test patient in the FHIR service. You'll need FHIR Data Contributor role.

In [ ]:
# Get access token for FHIR service
$token = (Get-AzAccessToken -ResourceUrl "https://azurehealthcareapis.com").Token

$headers = @{
    "Authorization" = "Bearer $token"
    "Content-Type" = "application/json"
}

# Patient resource
$patientResource = @{
    resourceType = "Patient"
    id = "1"
    name = @(
        @{
            family = "Patient1"
            given = @("Test")
        }
    )
} | ConvertTo-Json

try {
    $response = Invoke-RestMethod -Uri "$fhirUrl/Patient/1" -Method Put -Headers $headers -Body $patientResource
    Write-Host "✓ Patient resource created successfully" -ForegroundColor Green
    Write-Host "  Patient ID: $($response.id)" -ForegroundColor Cyan
    Write-Host "  Name: $($response.name[0].given[0]) $($response.name[0].family)" -ForegroundColor Cyan
} catch {
    Write-Host "✗ Failed to create patient: $_" -ForegroundColor Red
    Write-Host "  Ensure you have FHIR Data Contributor role on the FHIR service" -ForegroundColor Yellow
}

In [ ]:
# Verification: Retrieve the patient
try {
    $response = Invoke-RestMethod -Uri "$fhirUrl/Patient/1" -Method Get -Headers $headers
    Write-Host "✓ Patient retrieved successfully:" -ForegroundColor Green
    $response | ConvertTo-Json -Depth 5
} catch {
    Write-Host "✗ Failed to retrieve patient: $_" -ForegroundColor Red
}

## Step 7: Link Patient Resource to External ID User

Link the FHIR patient to the External ID user using Microsoft Graph.

In [ ]:
# Get b2c-extensions-app client ID
Connect-MgGraph -TenantId $externalTenantId -Scopes "Application.Read.All", "User.ReadWrite.All"

try {
    $b2cApp = Get-MgApplication -Filter "displayName eq 'b2c-extensions-app'"
    $b2cExtensionAppId = $b2cApp.AppId
    $b2cExtensionAppIdNoHyphens = $b2cExtensionAppId -replace '-', ''
    
    Write-Host "✓ b2c-extensions-app found:" -ForegroundColor Green
    Write-Host "  Client ID: $b2cExtensionAppId" -ForegroundColor Cyan
    Write-Host "  ID (no hyphens): $b2cExtensionAppIdNoHyphens" -ForegroundColor Cyan
    
    # Store for next step
    $global:b2cExtensionAppIdNoHyphens = $b2cExtensionAppIdNoHyphens
} catch {
    Write-Host "✗ Failed to find b2c-extensions-app: $_" -ForegroundColor Red
}

In [ ]:
# Update user with fhirUser extension attribute
$fhirUserValue = "$fhirUrl/Patient/1"
$extensionProperty = "extension_${b2cExtensionAppIdNoHyphens}_fhirUser"

$body = @{
    $extensionProperty = $fhirUserValue
}

try {
    Update-MgUser -UserId $testUserObjectId -BodyParameter $body
    Write-Host "✓ User linked to patient resource successfully" -ForegroundColor Green
    Write-Host "  User: $testUserPrincipalName" -ForegroundColor Cyan
    Write-Host "  Patient: $fhirUserValue" -ForegroundColor Cyan
    Write-Host "  Extension Property: $extensionProperty" -ForegroundColor Cyan
} catch {
    Write-Host "✗ Failed to link user to patient: $_" -ForegroundColor Red
}

In [ ]:
# Verification: Check user's extension attribute
try {
    $user = Get-MgUser -UserId $testUserObjectId -Property "id,displayName,userPrincipalName,*"
    $extensionValue = $user.AdditionalProperties[$extensionProperty]
    
    if ($extensionValue -eq $fhirUserValue) {
        Write-Host "✓ User-Patient link verified successfully" -ForegroundColor Green
        Write-Host "  fhirUser value: $extensionValue" -ForegroundColor Cyan
    } else {
        Write-Host "⚠️  fhirUser attribute not found or incorrect" -ForegroundColor Yellow
        Write-Host "  Expected: $fhirUserValue" -ForegroundColor Yellow
        Write-Host "  Found: $extensionValue" -ForegroundColor Yellow
    }
} catch {
    Write-Host "✗ Verification failed: $_" -ForegroundColor Red
}

## Step 8: Token Configuration Summary

Summary of OAuth2 endpoints and configuration for obtaining access tokens.

In [ ]:
# Display OAuth2 configuration for API testing tools (e.g., Insomnia, Postman)
Write-Host "=" * 80 -ForegroundColor Cyan
Write-Host "OAuth2 Configuration for API Testing" -ForegroundColor Green
Write-Host "=" * 80 -ForegroundColor Cyan

Write-Host "\nCallback URL:" -ForegroundColor Yellow
Write-Host "  http://localhost:3000" -ForegroundColor White

Write-Host "\nAuthorization URL:" -ForegroundColor Yellow
Write-Host "  https://$externalTenantName.ciamlogin.com/$externalTenantId/oauth2/v2.0/authorize" -ForegroundColor White

Write-Host "\nAccess Token URL:" -ForegroundColor Yellow
Write-Host "  https://$externalTenantName.ciamlogin.com/$externalTenantId/oauth2/v2.0/token" -ForegroundColor White

Write-Host "\nClient ID:" -ForegroundColor Yellow
Write-Host "  $appRegistrationClientId" -ForegroundColor White

Write-Host "\nScope:" -ForegroundColor Yellow
Write-Host "  api://$appRegistrationClientId/patient.all.read" -ForegroundColor White

Write-Host "\nFHIR Service URL:" -ForegroundColor Yellow
Write-Host "  $fhirUrl" -ForegroundColor White

Write-Host "\nTest User Credentials:" -ForegroundColor Yellow
Write-Host "  Username: $testUserPrincipalName" -ForegroundColor White
Write-Host "  Password: (the password you set during user creation)" -ForegroundColor White

Write-Host "\n" + "=" * 80 -ForegroundColor Cyan
Write-Host "\nNext Steps:" -ForegroundColor Green
Write-Host "1. Open your API testing tool (Insomnia, Postman, etc.)" -ForegroundColor Cyan
Write-Host "2. Configure OAuth 2.0 with the above values" -ForegroundColor Cyan
Write-Host "3. Get a new access token (this will trigger the user flow)" -ForegroundColor Cyan
Write-Host "4. Sign in with the test user credentials" -ForegroundColor Cyan
Write-Host "5. Use the token to make a GET request to: $fhirUrl/Patient" -ForegroundColor Cyan
Write-Host "6. Verify you receive only the linked patient resource (Patient/1)" -ForegroundColor Cyan

## Testing with PowerShell (Alternative)

If you prefer to test token acquisition in PowerShell (device code flow):

In [ ]:
# Note: This uses device code flow which may not trigger the full user flow with custom attributes
# Recommended to use Insomnia/Postman with the authorization code flow instead

Write-Host "⚠️  For full testing, use an API testing tool with authorization code flow" -ForegroundColor Yellow
Write-Host "Device code flow may not include custom claims like fhirUser" -ForegroundColor Yellow

# Example structure (not fully functional without interactive browser)
$scope = "api://$appRegistrationClientId/patient.all.read"
Write-Host "\nTo test manually, use the OAuth2 configuration above in an API testing tool." -ForegroundColor Cyan

## Summary and Verification Checklist

Review this checklist to ensure all steps were completed successfully.

In [ ]:
Write-Host "\n" + "=" * 80 -ForegroundColor Cyan
Write-Host "Setup Verification Checklist" -ForegroundColor Green
Write-Host "=" * 80 -ForegroundColor Cyan

$checklist = @(
    "☐ External ID tenant created and accessible",
    "☐ Test user created in External ID tenant",
    "☐ fhirUser custom attribute created",
    "☐ User flow created with fhirUser attribute",
    "☐ App registration created with OAuth2 permissions",
    "☐ Application ID URI configured",
    "☐ FHIR API permissions granted with admin consent",
    "☐ Token claims mapping enabled",
    "☐ SSO claims configured with fhirUser",
    "☐ FHIR Service deployed in primary tenant",
    "☐ Patient resource created in FHIR service",
    "☐ User linked to patient resource via fhirUser attribute",
    "☐ OAuth2 endpoints tested with API tool",
    "☐ Access token obtained successfully",
    "☐ FHIR patient query returns only linked patient"
)

foreach ($item in $checklist) {
    Write-Host "  $item" -ForegroundColor White
}

Write-Host "\n" + "=" * 80 -ForegroundColor Cyan
Write-Host "\nKey Resources:" -ForegroundColor Green
Write-Host "  External Tenant ID: $externalTenantId" -ForegroundColor Cyan
Write-Host "  App Registration Client ID: $appRegistrationClientId" -ForegroundColor Cyan
Write-Host "  FHIR Service URL: $fhirUrl" -ForegroundColor Cyan
Write-Host "  Test User: $testUserPrincipalName" -ForegroundColor Cyan
Write-Host "\n" + "=" * 80 -ForegroundColor Cyan